## Login access

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Installation process

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-taak9gy2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-taak9gy2
  Resolved https://github.com/huggingface/transformers to commit 78f6ed6c70b29c1560780e3869a7ad4c6b3d2710
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=7994863 sha256=5d2b0bfdc110034e75c5bcacefefc4964957d0153a4af59cdb6d34008ea412e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-a8th5kg1/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [ ]:
! pip install -q peft  accelerate bitsandbytes safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.3 MB/s eta 0:00:00


## Mistral 7B - Fine tuning

**Step 1: Retrieve the fine-tuned model and the pre-trained Mistral model**

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
adapters_name = "Baptiste33/mistral-7b-finetuned" # add your profile name of hugging-face and you fine_tuned model
model_name = "bn22/Mistral-7B-Instruct-v0.1-sharded" #"mistralai/Mistral-7B-Instruct-v0.1"


device = "cuda" # the device to load the model onto


**Step 2: Load base model Mistral 7B**

We will now load a model using 4-bit precision for faster training. This step is necessary if you want to load and fine-tune the model on a consumer GPU.

In [ ]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

This class is a part of the Transformers library and is used for causal language modeling tasks. "Causal language modeling" typically involves predicting the next word in a sequence, given the context of the preceding words.

This function loads a pre-trained model specified by its name. The argument "model_name" should be replaced with the actual name of the pre-trained model you want to use.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

 This is a method used to load a pre-trained model.

In [ ]:
model = PeftModel.from_pretrained(model, adapters_name)
#model = model.merge_and_unload()


Loads the pre-trained tokenizer. Convert a continuous text into individual tokens.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Successfully loaded the model bn22/Mistral-7B-Instruct-v0.1-sharded into memory


**Step 3: Evaluate the model**

In [ ]:

text = "[INST] generate a midjourney prompt for A person walks in the rain [/INST]"
# text = "[INST] generate a midjourney prompt for spiderman fighting villains  [/INST]"
# text = "[INST] What is your favourite condiment? [/INST]"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
model.to(device)
generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST] generate a midjourney prompt for A person walks in the rain [/INST] A person walks in the rain, lost in thought and lost in the pitter-patter of the raindrops. Suddenly, they notice something glinting in the corner of their eye. They pick up their pace, drawn to the shiny object. When they get closer, they see that it's an old, tarnished key. The person's mind races with possibilities. What door does the key unlock? What secrets does it hold? The person's journey takes a new turn, as they begin to follow the trail of the key, leading them deeper into the heart of the mysteries of the world.</s>


[INST] generate a midjourney prompt for A person walks in the rain [/INST] A person walks in the rain, lost in thought and lost in the pitter-patter of the raindrops. Suddenly, they notice something glinting in the corner of their eye. They pick up their pace, drawn to the shiny object. When they get closer, they see that it's an old, tarnished key. The person's mind races with possibilities. What door does the key unlock? What secrets does it hold? The person's journey takes a new turn, as they begin to follow the trail of the key, leading them deeper into the heart of the mysteries of the world.</s>


[INST] What is your favourite condiment? [/INST] My favorite condiment is sambal. (It's a spicy chili paste commonly used in Southeast Asian cuisine.)</s>

[INST] generate a midjourney prompt for spiderman fighting villains  [/INST] A villain has taken hostages and is holding them on a precarious building. Spiderman is the only person who can save them but must fight his way through the villain's army of minions and reach the hostages before it's too late. Along the way, he faces obstacles like treacherous flooring, fast-moving traps, and powerful foes.

As he makes his way towards the hostages, Spiderman encounters a villain who is not only powerful, but also seemingly invulnerable. Spiderman must use all of his skills and technology to defeat the villain and continue on his mission.

As he fights his way to the hostages, Spiderman discovers a surprising twist - the villain had a motive for kidnapping the hostages that goes beyond revenge or power. He reveals a personal vendetta that Spiderman had no idea about.

In the clim